In [ ]:
## weather_server.py
from typing import Any
import httpx
from textwrap import dedent
from mcp.server.fastmcp import FastMCP
import nest_asyncio
nest_asyncio.apply()


# Create an MCP server
mcp = FastMCP(name="weather",
            host="0.0.0.0",  # only used for SSE transport (localhost)
            port=8050,  # only used for SSE transport (set this to any port)
            )

# Constants
NWS_API_BASE = "https://api.weather.gov"
USER_AGENT = "weather-app/1.0"


async def make_nws_request(url: str) -> dict[str, Any] | None:
    """Make a request to the NWS API with proper error handling."""
    headers = {"User-Agent": USER_AGENT,
            "Accept": "application/geo+json"
            }
    
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json()
        except Exception:
            return None

def format_alert(feature: dict) -> str:
    """Format an alert feature into a readable string."""
    props = feature["properties"]
    result =  dedent(f"""
                Event: {props.get('event', 'Unknown')}
                Area: {props.get('areaDesc', 'Unknown')}
                Severity: {props.get('severity', 'Unknown')}
                Description: {props.get('description', 'No description available')}
                Instructions: {props.get('instruction', 'No specific instructions provided')}
            """)
    return result

@mcp.tool()
async def get_alerts(state: str) -> str:
    """Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    """
    url = f"{NWS_API_BASE}/alerts/active/area/{state}"
    data = await make_nws_request(url)

    if not data or "features" not in data:
        return "Unable to fetch alerts or no alerts found."

    if not data["features"]:
        return "No active alerts for this state."

    alerts = [format_alert(feature) for feature in data["features"]]
    return "\n---\n".join(alerts)


@mcp.tool()
async def get_forecast(latitude: float, longitude: float) -> str:
    """Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    """
    # First get the forecast grid endpoint
    points_url = f"{NWS_API_BASE}/points/{latitude},{longitude}"
    points_data = await make_nws_request(points_url)

    if not points_data:
        return "Unable to fetch forecast data for this location."

    # Get the forecast URL from the points response
    forecast_url = points_data["properties"]["forecast"]
    forecast_data = await make_nws_request(forecast_url)

    if not forecast_data:
        return "Unable to fetch detailed forecast."

    # Format the periods into a readable forecast
    periods = forecast_data["properties"]["periods"]
    forecasts = []
    for period in periods[:5]:  # Only show next 5 periods
        forecast = f"""
                {period['name']}:
                Temperature: {period['temperature']}°{period['temperatureUnit']}
                Wind: {period['windSpeed']} {period['windDirection']}
                Forecast: {period['detailedForecast']}
            """
        forecasts.append(forecast)

    return "\n---\n".join(forecasts)

# Run the server
if __name__ == "__main__":
    transport = "sse"
    if transport == "stdio":
        print("Running server with stdio transport")
        mcp.run(transport="stdio")
    elif transport == "sse":
        print("Running server with SSE transport")
        mcp.run(transport="sse")
    else:
        raise ValueError(f"Unknown transport: {transport}")

In [2]:
## client_sse.py
import asyncio
import nest_asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client

nest_asyncio.apply()  # Needed to run interactive python

"""
Make sure:
1. The server is running before running this script.
2. The server is configured to use SSE transport.
3. The server is listening on port 8050.

To run the server:
uv run server.py
"""

async def main(url):
    # Connect to the server using SSE
    async with sse_client(url) as (read_stream, write_stream):
        async with ClientSession(read_stream, write_stream) as session:
            # Initialize the connection
            await session.initialize()

            # List available tools
            tools_result = await session.list_tools()
            print("Available tools:")
            for tool in tools_result.tools:
                print(f"  - {tool.name}: {tool.description}")

            # Call our Weather tool
            result = await session.call_tool("get_alerts", arguments={"state":"CA"})
            print(f"The weather alerts are = {result.content[0].text}")


if __name__ == "__main__":
    url = "http://localhost:8050/sse"
    asyncio.run(main(url))

Connecting to SSE endpoint: http://localhost:8050/sse
HTTP Request: GET http://localhost:8050/sse "HTTP/1.1 200 OK"
Received endpoint URL: http://localhost:8050/messages/?session_id=fb974d7e9a9a4dadaa2e6999637a7299
Starting post writer with endpoint URL: http://localhost:8050/messages/?session_id=fb974d7e9a9a4dadaa2e6999637a7299
HTTP Request: POST http://localhost:8050/messages/?session_id=fb974d7e9a9a4dadaa2e6999637a7299 "HTTP/1.1 202 Accepted"
HTTP Request: POST http://localhost:8050/messages/?session_id=fb974d7e9a9a4dadaa2e6999637a7299 "HTTP/1.1 202 Accepted"
HTTP Request: POST http://localhost:8050/messages/?session_id=fb974d7e9a9a4dadaa2e6999637a7299 "HTTP/1.1 202 Accepted"
HTTP Request: POST http://localhost:8050/messages/?session_id=fb974d7e9a9a4dadaa2e6999637a7299 "HTTP/1.1 202 Accepted"


Available tools:
  - get_alerts: Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    
  - get_forecast: Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    
The weather alerts are = 
                Event: Flood Advisory
                Area: San Bernardino, CA
                Severity: Minor
                Description: * WHAT...Flooding caused by excessive rainfall continues.

* WHERE...Southern San Bernardino County including the Morongo Basin
and Yucca Valley.

* WHEN...Until 500 PM PDT.

* IMPACTS...Minor flooding in low-lying and poor drainage areas.
Water over roadways.

* ADDITIONAL DETAILS...
- At 247 PM PDT, Scattered showers and isolated thunderstorms
persist across the region. Minor flooding remains possible.
- Some locations that will experience flooding include...
Twentynine Palms, Yucca Valley, Twentynine Palms Base, Amboy,
Twentynin

## Run using Docker Steps:
 1. Build a docker container using the docker file
 2. Run the container with port tunnel with your local (9000:8050)
 3. it will run the server in 8050 as mentioned in docker file and expose the server in port 9000 in local 
 4. Now connect the mcp client with sse connection with the server running on the docker container
 5. Ask query and see mcp tool is invoked 

In [18]:
!docker build -t mcp_docker_demo:latest .

[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 454B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.11-slim        0.2s
[+] Building 0.3s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 454B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.11-slim        0.3s
[+] Building 0.5s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 454B                                       0.0s
 => [internal] load metadata for docker.

In [19]:
!docker images

REPOSITORY        TAG       IMAGE ID       CREATED         SIZE
mcp_docker_demo   latest    b4bbe569b4ed   4 minutes ago   360MB


In [21]:
!docker ps -a

CONTAINER ID   IMAGE          COMMAND              CREATED         STATUS                     PORTS     NAMES
c436ed585e14   70a2cda3b64c   "uv run server.py"   4 minutes ago   Exited (2) 4 minutes ago             cool_buck


In [22]:
!docker run -d -p 9000:8050 mcp_docker_demo

5fe65cfdd70c458fb57fd355bd1379ebb8995ee7374dd11165cfa0d1ac35c34a


In [23]:
## client_sse.py
import asyncio
import nest_asyncio
from mcp import ClientSession
from mcp.client.sse import sse_client

nest_asyncio.apply()  # Needed to run interactive python

"""
Make sure:
1. The server is running before running this script.
2. The server is configured to use SSE transport.
3. The server is listening on port 8050.

To run the server:
uv run server.py
"""

async def main(url):
    # Connect to the server using SSE
    async with sse_client(url) as (read_stream, write_stream):
        async with ClientSession(read_stream, write_stream) as session:
            # Initialize the connection
            await session.initialize()

            # List available tools
            tools_result = await session.list_tools()
            print("Available tools:")
            for tool in tools_result.tools:
                print(f"  - {tool.name}: {tool.description}")

            # Call our Weather tool
            result = await session.call_tool("get_alerts", arguments={"state":"CA"})
            print(f"The weather alerts are = {result.content[0].text}")


if __name__ == "__main__":
    url = "http://localhost:9000/sse"  ## we are accessing the server running in docker
    asyncio.run(main(url))

Connecting to SSE endpoint: http://localhost:9000/sse
HTTP Request: GET http://localhost:9000/sse "HTTP/1.1 200 OK"
Received endpoint URL: http://localhost:9000/messages/?session_id=6e3b326439994df0843d01b08790b79a
Starting post writer with endpoint URL: http://localhost:9000/messages/?session_id=6e3b326439994df0843d01b08790b79a
HTTP Request: POST http://localhost:9000/messages/?session_id=6e3b326439994df0843d01b08790b79a "HTTP/1.1 202 Accepted"
HTTP Request: POST http://localhost:9000/messages/?session_id=6e3b326439994df0843d01b08790b79a "HTTP/1.1 202 Accepted"
HTTP Request: POST http://localhost:9000/messages/?session_id=6e3b326439994df0843d01b08790b79a "HTTP/1.1 202 Accepted"
HTTP Request: POST http://localhost:9000/messages/?session_id=6e3b326439994df0843d01b08790b79a "HTTP/1.1 202 Accepted"


Available tools:
  - get_alerts: Get weather alerts for a US state.

    Args:
        state: Two-letter US state code (e.g. CA, NY)
    
  - get_forecast: Get weather forecast for a location.

    Args:
        latitude: Latitude of the location
        longitude: Longitude of the location
    
The weather alerts are = 
                Event: Flood Advisory
                Area: San Bernardino, CA
                Severity: Minor
                Description: * WHAT...Flooding caused by excessive rainfall continues.

* WHERE...Southern San Bernardino County including the Morongo Basin
and Yucca Valley.

* WHEN...Until 500 PM PDT.

* IMPACTS...Minor flooding in low-lying and poor drainage areas.
Water over roadways.

* ADDITIONAL DETAILS...
- At 247 PM PDT, Scattered showers and isolated thunderstorms
persist across the region. Minor flooding remains possible.
- Some locations that will experience flooding include...
Twentynine Palms, Yucca Valley, Twentynine Palms Base, Amboy,
Twentynin